In [2]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

##### Replicating Liu et al 2019

In [62]:
data = pd.read_pickle('/mnt/BigDisk18T/Hrq/working/Liu_2019/monthly_dataset.pkl')  # Data from Wind

data['tot_shr'] = data['tot_shr'] * data['price']
data['mktcap_lastm'] = data.groupby(['stkcd'])['tot_shr'].shift(1)


data['lg_ret'] = np.log(1+data['ret'])
data['pre12-month reversal'] = (np.exp(data.groupby(['stkcd'])['lg_ret'].rolling(12,min_periods=1).sum())-1).reset_index(level=0)['lg_ret']
data['pre12-month reversal'] = data.groupby(['stkcd'])['pre12-month reversal'].shift(1)

data['pre1-month reversal'] = data.groupby(['stkcd'])['ret'].shift(1)
data.drop(columns=['lg_ret'],inplace=True)



# filter
# no new stock
data['listdt'] = data['listdt'] // 100


# least trading day constraint
data['past12month_trddate'] = data.groupby(['stkcd'])['num_day'].rolling(12,min_periods = 1).sum().reset_index(level=0)['num_day']
data['past12month_trddate'] = data.groupby(['stkcd'])['past12month_trddate'].shift(1)
data['past1month_trddate'] = data.groupby(['stkcd'])['num_day'].shift(1)


data = data[((data['month'] - data['listdt'])>=6)].copy()
data = data[(data['past12month_trddate']>=120) ].copy()  #& (data['past1month_trddate']>=15)
data = data[((data['stkcd']//10000).isin([0,30,60])) & (data['month']>=199801) & (data['month']<=202201)].copy()

# drop smallest 30p
tp = data.groupby(['month'])['mktcap_lastm'].quantile(0.3).reset_index()
tp.columns = ['month','smallest30p']

data = pd.merge(data,tp,on=['month'],how='left')
del tp
data = data[(data['mktcap_lastm'] >= data['smallest30p'])].copy()


data.drop(columns=['past12month_trddate','past1month_trddate','smallest30p'],inplace=True)

In [66]:
data['Size'] = data['tot_shr']
data['EP'] = data['netprofit'] / data['tot_shr']
data['BM'] = data['TOT_SHRHLDR_EQY_EXCL_MIN_INT'] / data['tot_shr']
data['CP'] = data['CASH'] / data['tot_shr']
data['ROE'] = data['netprofit'] / data['TOT_SHRHLDR_EQY_EXCL_MIN_INT']

tp = data[['stkcd','ym','Size','EP','BM','CP','ROE']].copy()
tp.columns = ['stkcd','ym','preSize','preEP','preBM','preCP','preROE']
tp['ym'] += 1
data = pd.merge(data,tp,on=['stkcd','ym'],how='left')
del tp

In [67]:
# SMB  VMG of ch3

tp = data.groupby(['month'])['preSize'].median().reset_index(level=0)
tp.columns = ['month','SB']

data = pd.merge(data,tp,on=['month'],how='left')
del tp


tp = data.groupby(['month'])['preEP'].quantile([0.3,0.7]).unstack().reset_index()
tp.columns = ['month','G','V']

data = pd.merge(data,tp,on=['month'],how='left')
del tp


data['SB'] = np.where(data['preSize']>=data['SB'],'B','S')
data['VG'] = np.where(data['preEP']>=data['V'],'V','M')
data['VG'] = np.where(data['preEP']<data['G'],'G',data['VG'])

data.loc[data['preSize'].isna(),'SB'] = np.NAN
data.loc[data['preEP'].isna(),'VG'] = np.NAN

data['type'] = data['SB'] + data['VG']

res_rep = data.dropna(subset=['SB','VG']).groupby(['month','type']).apply(lambda x:(x['mktcap_lastm']*x['ret']).sum() / x['mktcap_lastm'].sum()).unstack().loc[200201:]
MKT = data.groupby(['month']).apply(lambda x:(x['mktcap_lastm']*x['ret']).sum() / x['mktcap_lastm'].sum()).loc[200201:] *100
SMB = ((res_rep['SV'] + res_rep['SM'] + res_rep['SG'])/3 - (res_rep['BG']+res_rep['BM']+res_rep['BV'])/3) * 100
VMG = ((res_rep['SV'] + res_rep['BV'])/2 - (res_rep['SG'] + res_rep['BG'])/2) * 100
ch3 = pd.read_csv('/mnt/BigDisk18T/Hrq/CH_3_update_20211231.csv')
ch3['mnthdt'] = ch3['mnthdt']//100
ch3.index = ch3['mnthdt']
ch3.drop(columns=['mnthdt'],inplace=True)
ch3['MKT_my'] = MKT
ch3['SMB_my'] = SMB
ch3['VMG_my'] = VMG
ch3['MKT_my'] = ch3['MKT_my'] - ch3['rf_mon']
ch3.loc[200201:201612].corr()
